In [4]:
from torch_geometric.datasets import Planetoid, Twitch, TUDataset, Amazon
import numpy as np
import torch

In [5]:
# Load Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

# Calculate class proportions
labels = data.y.numpy()
unique, counts = np.unique(labels, return_counts=True)
N = labels.size
alpha_c = counts / N

# Calculate actual accuracy (Placeholder for simulation or actual calculation)
# This part requires an actual link prediction model or attack model which we're approximating.
# attack_accuracy = simulate_attack(data)  # This would be an actual function if we had it.

# Calculate approximated accuracy
approximated_accuracy = 1 - np.sum(alpha_c**2)

print("Class Proportions (alpha_c):", alpha_c)
print("Approximated Attack Accuracy:", approximated_accuracy)


Class Proportions (alpha_c): [0.12961595 0.08013294 0.15435746 0.30206795 0.15731167 0.11004431
 0.06646972]
Approximated Attack Accuracy: 0.8204322113590892


In [46]:
import numpy as np
from scipy.optimize import minimize

def optimize_proportions(initial_proportions, budget, max_iter=1000):
    n = len(initial_proportions)
    
    # Objective function (negative sum of squares)
    def objective(alpha):
        return -np.sum(alpha**2)
    
    # Equality constraint: sum of proportions must equal 1
    def equality_constraint(alpha):
        return np.sum(alpha) - 1
    
    # Inequality constraint: sum of negative changes must be within budget
    def inequality_constraint(alpha):
        return budget - np.sum(np.maximum(0, initial_proportions - alpha))
    
    # Non-negativity constraint
    bounds = [(0, None) for _ in range(n)]
    
    # Initial guess (starting point)
    initial_guess = np.array(initial_proportions)
    
    # Define constraints in the form required by `scipy.optimize.minimize`
    constraints = [
        {'type': 'eq', 'fun': equality_constraint},
        {'type': 'ineq', 'fun': inequality_constraint}
    ]
    
    # Perform the optimization with increased iteration limit
    result = minimize(
        objective, 
        initial_guess, 
        method='SLSQP', 
        bounds=bounds, 
        constraints=constraints,
        options={'maxiter': max_iter, 'disp': True}
    )
    
    # Print debugging information
    print("Optimization Result:", result)
    
    if result.success:
        optimized_proportions = result.x
        optimized_sum_of_squares = -result.fun  # Since we minimized the negative sum of squares
        return optimized_proportions, optimized_sum_of_squares
    else:
        raise ValueError("Optimization failed: " + result.message)

# Initial proportions and budget for debugging
initial_proportions = [0.12961595, 0.08013294, 0.15435746, 0.30206795, 0.15731167, 0.11004431, 0.06646972]
budget = 0.2

# Run the optimization
try:
    optimized_proportions, optimized_sum_of_squares = optimize_proportions(initial_proportions, budget)
    print("Optimized Proportions:", optimized_proportions)
    print("Optimized Sum of Squares:", optimized_sum_of_squares)
except ValueError as e:
    print(e)


def convert_labels(initial_labels, initial_proportions, optimized_proportions):
    n = len(initial_proportions)
    num_nodes = len(initial_labels)
    
    # Calculate the total number of nodes for each label
    initial_counts = np.array([np.sum(np.array(initial_labels) == i) for i in range(n)])
    optimized_counts = np.round(num_nodes * optimized_proportions).astype(int)
    
    # Adjust counts to make sure the total number of nodes remains the same
    difference = num_nodes - np.sum(optimized_counts)
    if difference > 0:
        # If the sum of optimized counts is less, add the difference to the largest proportion
        max_index = np.argmax(optimized_counts)
        optimized_counts[max_index] += difference
    elif difference < 0:
        # If the sum of optimized counts is more, subtract the difference from the largest proportion
        max_index = np.argmax(optimized_counts)
        optimized_counts[max_index] += difference
    
    # Find out how many nodes need to be moved from each label
    changes = optimized_counts - initial_counts
    
    # Initialize the new labels array
    new_labels = np.array(initial_labels)
    
    # Indices of nodes for each label
    label_indices = [np.where(np.array(initial_labels) == i)[0] for i in range(n)]
    
    for i in range(n):
        if changes[i] < 0:
            # If the count is negative, we need to move nodes out of this label
            excess_indices = label_indices[i][:abs(changes[i])]
            for j in range(n):
                if changes[j] > 0:
                    # Move nodes to labels with positive changes
                    num_to_move = min(abs(changes[i]), changes[j])
                    new_labels[excess_indices[:num_to_move]] = j
                    excess_indices = excess_indices[num_to_move:]
                    changes[j] -= num_to_move
                    if len(excess_indices) == 0:
                        break
            changes[i] = 0
    
    return new_labels

# Example usage
initial_labels = data.y
initial_proportions = alpha_c
budget = 0.3

optimized_proportions, _ = optimize_proportions(initial_proportions, budget)
new_labels = convert_labels(initial_labels, initial_proportions, optimized_proportions)

print("Initial Labels:", np.array(initial_labels))
print("Optimized Proportions:", np.array(optimized_proportions))
print("New Labels:", new_labels)


Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.2516645430416846
            Iterations: 53
            Function evaluations: 524
            Gradient evaluations: 51
Optimization Result:      fun: -0.2516645430416846
     jac: array([-6.59208618e-01, -1.13383174e-01, -3.08625728e-01, -6.04159206e-01,
       -3.14623337e-01, -1.49011612e-08, -1.49011612e-08])
 message: 'Optimization terminated successfully'
    nfev: 524
     nit: 53
    njev: 51
  status: 0
 success: True
       x: array([0.3296065 , 0.05669163, 0.15431281, 0.3020774 , 0.15731166,
       0.        , 0.        ])
Optimized Proportions: [0.3296065  0.05669163 0.15431281 0.3020774  0.15731166 0.
 0.        ]
Optimized Sum of Squares: 0.2516645430416846
Iteration limit reached    (Exit mode 9)
            Current function value: -0.4185013911940551
            Iterations: 1000
            Function evaluations: 17805
            Gradient evaluations: 1000
Optimization Result:   

ValueError: Optimization failed: Iteration limit reached

(array([0, 1, 2, 3, 4, 5]), array([ 351,  126,  418, 1089,  426,  298]))

In [29]:
print("Initial Labels:", np.array(initial_labels))
print("Optimized Proportions:", np.array(optimized_proportions))
print("New Labels:", new_labels)
print("Initial Labels:", np.array(initial_labels))

print(np.unique(initial_labels, return_counts=True))

Initial Labels: [3 4 4 ... 3 3 3]
Optimized Proportions: [1.29615814e-01 4.66028307e-02 1.54357399e-01 4.02068893e-01
 1.57310764e-01 1.10044299e-01 9.37894406e-18]
New Labels: [3 4 4 ... 3 3 3]
Initial Labels: [3 4 4 ... 3 3 3]
(array([0, 1, 2, 3, 4, 5, 6]), array([351, 217, 418, 818, 426, 298, 180]))


In [28]:
np.unique(new_labels, return_counts=True)

(array([0, 1, 2, 3, 4, 5]), array([ 351,  126,  418, 1089,  426,  298]))

In [27]:
initial_proportions.round(3)

array([0.13 , 0.08 , 0.154, 0.302, 0.157, 0.11 , 0.066])

In [26]:
optimized_proportions.round(3)

array([0.13 , 0.047, 0.154, 0.402, 0.157, 0.11 , 0.   ])

In [34]:
np.sum(new_labels != initial_labels.numpy())

271

Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.4185013060521942
            Iterations: 28
            Function evaluations: 302
            Gradient evaluations: 27
Optimization Result:      fun: -0.4185013060521942
     jac: array([-1.72523271e-01, -1.49011612e-08, -3.08714963e-01, -1.20413847e+00,
       -3.14623352e-01, -1.49011612e-08, -1.49011612e-08])
 message: 'Optimization terminated successfully'
    nfev: 302
     nit: 28
    njev: 27
  status: 0
 success: True
       x: array([8.62619626e-02, 5.17817320e-12, 1.54357438e-01, 6.02068934e-01,
       1.57311666e-01, 1.67210469e-14, 3.48157125e-12])
Optimized Proportions: [8.62619626e-02 5.17817320e-12 1.54357438e-01 6.02068934e-01
 1.57311666e-01 1.67210469e-14 3.48157125e-12]
Optimized Sum of Squares: 0.4185013060521942


In [33]:
import numpy as np
import random
from scipy.optimize import minimize

def optimize_proportions(initial_proportions, budget, max_iter=1000):
    n = len(initial_proportions)
    
    # Objective function (negative sum of squares)
    def objective(alpha):
        return -np.sum(alpha**2)
    
    # Equality constraint: sum of proportions must equal 1
    def equality_constraint(alpha):
        return np.sum(alpha) - 1
    
    # Inequality constraint: sum of negative changes must be within budget
    def inequality_constraint(alpha):
        return budget - np.sum(np.maximum(0, initial_proportions - alpha))
    
    # Non-negativity constraint
    bounds = [(0, None) for _ in range(n)]
    
    # Initial guess (starting point)
    initial_guess = np.array(initial_proportions)
    
    # Define constraints in the form required by `scipy.optimize.minimize`
    constraints = [
        {'type': 'eq', 'fun': equality_constraint},
        {'type': 'ineq', 'fun': inequality_constraint}
    ]
    
    # Perform the optimization with increased iteration limit
    result = minimize(
        objective, 
        initial_guess, 
        method='SLSQP', 
        bounds=bounds, 
        constraints=constraints,
        options={'maxiter': max_iter, 'disp': True}
    )
    
    # Print debugging information
    print("Optimization Result:", result)
    
    if result.success:
        optimized_proportions = result.x
        optimized_sum_of_squares = -result.fun  # Since we minimized the negative sum of squares
        return optimized_proportions, optimized_sum_of_squares
    else:
        raise ValueError("Optimization failed: " + result.message)


import numpy as np
from collections import Counter

def balance_labels(current_labels, optimized_proportions):
    """
    Implement the BalanceLabels algorithm to redistribute labels based on optimized proportions.
    
    Args:
    current_labels (np.array): Current labels of nodes
    optimized_proportions (np.array): Optimized proportions for each label
    
    Returns:
    np.array: New labels after redistribution
    """
    total_nodes = len(current_labels)
    
    # Convert proportions to rounded counts
    optimized_counts = proportions_to_rounded_counts(optimized_proportions, total_nodes)
    
    # Count current labels using NumPy
    unique_labels, label_counts = np.unique(current_labels, return_counts=True)
    label_counts = dict(zip(unique_labels, label_counts))
    
    # Calculate in and out for each label
    label_changes = {}
    for i, label in enumerate(unique_labels):
        current_count = label_counts.get(label, 0)
        optimized_count = optimized_counts[i]
        difference = optimized_count - current_count
        label_changes[label] = {"in": max(0, difference), "out": max(0, -difference)}
    
    # Create a list of labels sorted by 'in' count (descending)
    labels_needing_nodes = sorted(label_changes.keys(), key=lambda x: label_changes[x]['in'], reverse=True)
    
    # Keep track of flipped nodes
    flipped_nodes = set()
    new_labels = current_labels.copy()
    
    # Iterate over labels needing nodes
    for target_label in labels_needing_nodes:
        nodes_needed = label_changes[target_label]['in']
        if nodes_needed == 0:
            continue
        
        # Iterate over labels with excess nodes
        for source_label in label_changes.keys():
            if source_label == target_label:
                continue
            nodes_available = label_changes[source_label]['out']
            if nodes_available == 0:
                continue
            
            # Determine number of nodes to flip
            nodes_to_flip = min(nodes_needed, nodes_available)
            
            # Find eligible nodes to flip
            eligible_nodes = [i for i, label in enumerate(new_labels) 
                              if label == source_label and i not in flipped_nodes]
            nodes_to_flip = min(nodes_to_flip, len(eligible_nodes))
            
            # Flip the labels
            for node in random.sample(eligible_nodes, nodes_to_flip):
                new_labels[node] = target_label
                flipped_nodes.add(node)
            
            # Update counts
            label_changes[target_label]['in'] -= nodes_to_flip
            label_changes[source_label]['out'] -= nodes_to_flip
            
            nodes_needed -= nodes_to_flip
            if nodes_needed == 0:
                break
    
    return new_labels

def proportions_to_rounded_counts(proportions, total_nodes):
    """
    Convert proportions to rounded counts while ensuring the total sum equals total_nodes.
    
    Args:
    proportions (np.array): A NumPy array of label proportions
    total_nodes (int): The total number of nodes in the graph
    
    Returns:
    np.array: An array of integer counts that sum to total_nodes
    """
    raw_counts = proportions * total_nodes
    rounded_counts = np.floor(raw_counts).astype(int)
    difference = total_nodes - np.sum(rounded_counts)
    sorted_indices = np.argsort(raw_counts - np.floor(raw_counts))[::-1]
    
    for i in range(int(difference)):
        rounded_counts[sorted_indices[i % len(sorted_indices)]] += 1
    
    return rounded_counts



In [27]:
# Initial proportions and budget for debugging
initial_proportions = [0.12961595, 0.08013294, 0.15435746, 0.30206795, 0.15731167, 0.11004431, 0.06646972]
N = 2708
budget = 0.1
# Run the optimization
try:
    optimized_proportions, optimized_sum_of_squares = optimize_proportions(initial_proportions, budget)
    print("Optimized Proportions:", optimized_proportions)
    print("Optimized Sum of Squares:", optimized_sum_of_squares)
except ValueError as e:
    print(e)
# Example usage
current_labels = data.y.tolist()
# optimized_proportions = np.array([0.4, 0.3, 0.3])  # Proportions for labels 1, 2, 3

new_labels = balance_labels(current_labels, optimized_proportions)

print("Original labels:", current_labels)
print("New labels:     ", new_labels)
print("Old label counts:", {label: current_labels.count(label) for label in set(current_labels)})
print("New label counts:", {label: new_labels.count(label) for label in set(new_labels)})
new_labels = torch.tensor(new_labels)

Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.24131410970168074
            Iterations: 34
            Function evaluations: 326
            Gradient evaluations: 34
Optimization Result:      fun: -0.24131410970168074
     jac: array([-2.59231504e-01, -9.32059586e-02, -3.08714814e-01, -8.04137828e-01,
       -3.14621560e-01, -2.20088420e-01, -1.49011612e-08])
 message: 'Optimization terminated successfully'
    nfev: 326
     nit: 34
    njev: 34
  status: 0
 success: True
       x: array([1.29615811e-01, 4.66028341e-02, 1.54357400e-01, 4.02068895e-01,
       1.57310766e-01, 1.10044294e-01, 4.77652364e-16])
Optimized Proportions: [1.29615811e-01 4.66028341e-02 1.54357400e-01 4.02068895e-01
 1.57310766e-01 1.10044294e-01 4.77652364e-16]
Optimized Sum of Squares: 0.24131410970168074
Original labels: [3, 4, 4, 0, 3, 2, 0, 3, 3, 2, 0, 0, 4, 3, 3, 3, 2, 3, 1, 3, 5, 3, 4, 6, 3, 3, 6, 3, 2, 4, 3, 6, 0, 4, 2, 0, 1, 5, 4, 4, 3, 6, 6, 4, 3, 3, 2, 5,

In [28]:
(new_labels != data.y).sum()

tensor(271)

In [29]:
180 + 217 - 126

271

In [30]:
def labels_defense(labels, budget):
    #clalculate the class proportions
    labels = labels.numpy()
    unique, counts = np.unique(labels, return_counts=True)
    N = labels.size
    alpha_c = counts / N
    #optimize the proportions
    try:
        optimized_proportions, optimized_sum_of_squares = optimize_proportions(alpha_c, budget)
        print("Optimized Proportions:", optimized_proportions)
        print("Optimized Sum of Squares:", optimized_sum_of_squares)
    except ValueError as e:
        print(e)
    #balance the labels
    new_labels = balance_labels(labels, optimized_proportions)
    return torch.tensor(new_labels)

In [49]:
new_labels = labels_defense(data.y, 0.05).tolist()
print("New label counts:", {label: new_labels.count(label) for label in set(new_labels)})

Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.20808105670318988
            Iterations: 50
            Function evaluations: 491
            Gradient evaluations: 48
Optimization Result:      fun: -0.20808105670318988
     jac: array([-0.25922666, -0.16024482, -0.30872221, -0.70414317, -0.31461408,
       -0.21958992, -0.03345924])
 message: 'Optimization terminated successfully'
    nfev: 491
     nit: 50
    njev: 48
  status: 0
 success: True
       x: array([0.12961391, 0.08012365, 0.15435654, 0.35206886, 0.15730833,
       0.10980679, 0.01672191])
Optimized Proportions: [0.12961391 0.08012365 0.15435654 0.35206886 0.15730833 0.10980679
 0.01672191]
Optimized Sum of Squares: 0.20808105670318988
New label counts: {0: 351, 1: 217, 2: 418, 3: 954, 4: 426, 5: 297, 6: 45}
